In [1]:
"""生体医工学ウェブ辞典（第一分冊・第二分冊）
cf. http://jsmbe.org/cyclopedia.html

医療分野の大規模テキストデータで学習した分散表現
cf. https://yag-ays.github.io/project/icd10-embeddings/

"""
import re
import os
import collections
import pandas as pd
from pprint import pprint

from termextract import LRValue
from termextract.preprocessing import MeCabTokenizer

In [2]:
file_path = "datasets/生体医工学ウェブ辞典用語リスト.csv"
df = pd.read_csv(file_path, encoding='utf-8')

In [3]:
df.head()

,NO,ヨミ,用語名称
0,1,B,BRS・Bioresorbable vascular scaffold・生体吸収性スキャフォールド
1,2,J,Judkinsカテーテル
2,3,あ,f-TUL・r-TUL・経尿道的尿管砕石術
3,4,あ,iPS細胞・人工多能性幹細胞
4,5,あ,アナログ信号・アナログ信号処理


In [4]:
idx_ = dict()  # id => term-list 
ivt_ = dict() # term=> id   
for index, row in df.iterrows():
    id = f"{row['NO']:03d}"
    terms = row['用語名称'].split("・")
    idx_[id] = terms
    # print(id, terms)
    for term in terms:
       ivt_[term.strip()] = id     

list(idx_.items())[:10]

[('001', ['BRS', 'Bioresorbable vascular scaffold', '生体吸収性スキャフォールド']),
 ('002', ['Judkinsカテーテル']),
 ('003', ['f-TUL', 'r-TUL', '経尿道的尿管砕石術']),
 ('004', ['iPS細胞', '人工多能性幹細胞']),
 ('005', ['アナログ信号', 'アナログ信号処理']),
 ('006', ['アナログ電気回路']),
 ('007', ['アンプ', '生体アンプ']),
 ('008', ['多極集積化アンプ', 'アナログ集積化アンプ']),
 ('009', ['ES細胞']),
 ('010', ['イメージベーストシミュレーション'])]

In [5]:
list(ivt_.items())[:10]

[('BRS', '001'),
 ('Bioresorbable vascular scaffold', '001'),
 ('生体吸収性スキャフォールド', '001'),
 ('Judkinsカテーテル', '002'),
 ('f-TUL', '003'),
 ('r-TUL', '003'),
 ('経尿道的尿管砕石術', '003'),
 ('iPS細胞', '004'),
 ('人工多能性幹細胞', '004'),
 ('アナログ信号', '005')]

In [6]:
item = '経尿道的尿管砕石術'
idx_[ivt_[item]]

['f-TUL', 'r-TUL', '経尿道的尿管砕石術']

In [7]:
item = 'BRS'
idx_[ivt_[item]]

['BRS', 'Bioresorbable vascular scaffold', '生体吸収性スキャフォールド']

In [8]:
tok = MeCabTokenizer() 
def wakati(text, tok):
    if text.isascii():
        return [[text]]

    tokens = tok.iter_token(text)
    # nouns = [token.surface for token in tokens]
    nouns = tok.extract_nouns(tokens)
    if len(nouns) > 0:
        return nouns
        
    return []

In [16]:
for term, _ in ivt_.items():
    nouns = wakati(term, tok)
    # pprint(nouns)
    if len(nouns)==1 and len(nouns[0])==1:#固有名詞
        print(nouns[0][0])
    elif len(nouns) > 0:#非固有名詞複合語
        m = sum([len(x) for x in nouns])
        print(f"({m})", " ".join(["_".join(n) for n in nouns]))       


BRS
Bioresorbable vascular scaffold
(4) 生体_吸収_性_スキャフォールド
(2) Judkins_カテーテル
f-TUL
r-TUL
(6) 経_尿道_的_尿管_砕石_術
iPS細胞
人工多能性幹細胞
(2) アナログ_信号
アナログ信号処理
(2) アナログ_電気回路
アンプ
(2) 生体_アンプ
(5) 多_極_集積_化_アンプ
(4) アナログ_集積_化_アンプ
ES細胞
イメージベーストシミュレーション
(2) 医用_画像
(2) 医療_情報
画像処理
画像解析
高分子
(2) 医用_高分子
統計
(2) 医学_統計
(2) 数理_統計学
膜電位
オンチャネル
活動電位
(2) VOF_法
(4) Volume_Of_Flud_法
Volume Of Flud
ウィンドケッセルモデル
(2) AD_変換
(2) DA_変換
LDD
Local Drug Delivery
(3) 局所_薬物_投与
(3) N_-_イソプロピルアクリルアミド
NIPAM
SHD
Structural Heart Disease
omics
オミックス
オキシメータ
オペアンプ
応力
CABG
(3) 冠動脈_バイパス_術
カテーテルアブレーション
(2) 活動電位_伝播
(2) 興奮_伝播
(3) 株_化_細胞
(3) 冠動脈_形成_術
(2) 冠動脈_造影
間葉系幹細胞
義肢
装具
虚血性心疾患
境界条件
(3) 局所_集合_電位
(3) 筋_音_図
(3) 筋_音_計
筋電図
筋電計
EMG
PCL
Percutaneous Coronary Intervention
経皮的冠動脈形成術
経カテーテル的大動脈弁置換術
TAVI
(3) 継_代_培養
血管造影
(7) 抗_血小板_薬_二_剤_併用_療法
高分子ミセル
(2) サンプリング_定理
再生医療
(2) 細_動
心室細動
心房細動
(2) システム_生理学
シナプス
(2) 自律_神経
交感神経
(2) 副_交感神経
(2) 受動_計測
(2) 能動_計測
(3) 集中_定数_モデル
循環系
(2) 初代_培養細胞
(3) 上流工程 上_工程
(4) 下流_工程 下_工程
心臓
(2) 心_機能
(3) 心臓_カテーテル_シミュレーター
(2) 神経_補綴
(2) 人工_視覚


In [17]:
import pylcs

In [48]:
X = "人工"
for term, _ in ivt_.items():
    nouns = wakati(term, tok)
    # pprint(nouns)
    Y = None
    if len(nouns)==1 and len(nouns[0])==1:#固有名詞
        Y = nouns[0][0]
    elif len(nouns) > 0:#非固有名詞複合語
        m = sum([len(x) for x in nouns])
        Y = "".join(["".join(n) for n in nouns]) 

    if Y:
        d = pylcs.lcs_sequence_length(X, Y)
        r = d/max(len(X),len(Y))
        if d >= len(X):
            print(f"{Y} ({d}): {r:.3f}")

人工多能性幹細胞 (2): 0.250
人工視覚 (2): 0.500
人工知能 (2): 0.500
人工網膜 (2): 0.500
人工括約筋 (2): 0.400
人工腎臓 (2): 0.500
人工食道 (2): 0.500
人工心筋 (2): 0.500
ナノテク人工心筋 (2): 0.250
人工心臓 (2): 0.500
全人工心臓 (2): 0.400
補助人工心臓 (2): 0.333
人工心肺システム (2): 0.250
人工生体組織 (2): 0.333
人工感覚器 (2): 0.400
人工臓器 (2): 0.500
人工透析 (2): 0.500
体外式膜型人工肺 (2): 0.250


In [54]:
for term, id in ivt_.items():
    nouns = wakati(term, tok)
    word = None
    if len(nouns)==1 and len(nouns[0])==1:#固有名詞
        word = nouns[0][0]
    elif len(nouns) > 0:#非固有名詞複合語
        m = sum([len(x) for x in nouns])
        word = ", ".join([".".join(n) for n in nouns]) 
    if word:
        print(id, word) 

001 BRS
001 Bioresorbable vascular scaffold
001 生体.吸収.性.スキャフォールド
002 Judkins.カテーテル
003 f-TUL
003 r-TUL
003 経.尿道.的.尿管.砕石.術
004 iPS細胞
004 人工多能性幹細胞
005 アナログ.信号
005 アナログ信号処理
006 アナログ.電気回路
007 アンプ
007 生体.アンプ
008 多.極.集積.化.アンプ
008 アナログ.集積.化.アンプ
009 ES細胞
010 イメージベーストシミュレーション
011 医用.画像
012 医療.情報
012 画像処理
012 画像解析
013 高分子
013 医用.高分子
014 統計
014 医学.統計
014 数理.統計学
015 膜電位
015 オンチャネル
015 活動電位
016 VOF.法
016 Volume.Of.Flud.法
016 Volume Of Flud
017 ウィンドケッセルモデル
018 AD.変換
018 DA.変換
019 LDD
019 Local Drug Delivery
019 局所.薬物.投与
020 N.-.イソプロピルアクリルアミド
020 NIPAM
021 SHD
021 Structural Heart Disease
022 omics
022 オミックス
023 オキシメータ
024 オペアンプ
025 応力
026 CABG
026 冠動脈.バイパス.術
027 カテーテルアブレーション
028 活動電位.伝播
028 興奮.伝播
029 株.化.細胞
030 冠動脈.形成.術
031 冠動脈.造影
032 間葉系幹細胞
033 義肢
033 装具
034 虚血性心疾患
035 境界条件
036 局所.集合.電位
037 筋.音.図
037 筋.音.計
038 筋電図
038 筋電計
038 EMG
039 PCL
039 Percutaneous Coronary Intervention
039 経皮的冠動脈形成術
040 経カテーテル的大動脈弁置換術
040 TAVI
041 継.代.培養
042 血管造影
043 抗.血小板.薬.二.剤.併用.療法
044 高分子ミセル
045 サンプリング.定理
046 再生医療
047 細.